In [8]:
import pandas as pd

In [9]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

In [10]:
df_train.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,...,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing,imdb_score,award_score,top_score,success_label
0,Stepmom's Desire,2020-01-01,movie,78.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Lee Soo, Tae Hee, James, Jung In",KR,...,7.150,20,"Sang-jin, who was envious of her neighbor afte...",1,0,0,21.768335,0.0,0.0,1
1,Pizza Dare 1,2020-01-01,movie,68.0,1.0,1.0,Romance,Kong Ja-kwan,"Yoo Jung, Yoon So-jung, Jin Si-ah, Baek Yoon-j...",KR,...,6.222,9,"Seok, who wants to try perverted sex, pesters ...",1,0,0,14.326684,0.0,0.0,0
2,Bosomy Mom,2020-01-01,movie,69.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Yoo Jung, Han Yi-seul, James, Shin Yeon-woo",KR,...,5.900,11,Da-hee gets married to Min-soo who is older th...,1,0,0,14.660949,0.0,0.0,0
3,Sonic the Hedgehog,2020-01-01,movie,99.0,1.0,1.0,"Action, Science Fiction, Comedy, Family",Jeff Fowler,"Ben Schwartz, James Marsden, Tika Sumpter, Jim...",US,...,7.307,9955,"Powered with incredible speed, Sonic The Hedge...",0,0,0,67.267735,0.0,0.0,1
4,Demon Slayer -Kimetsu no Yaiba- The Movie: Mug...,2020-01-01,movie,117.0,1.0,1.0,"Animation, Action, Fantasy, Thriller",Haruo Sotozaki,"Natsuki Hanae, Akari Kito, Hiro Shimono, Yoshi...",JP,...,8.209,4103,"Tanjiro Kamado, joined with Inosuke Hashibira,...",0,0,0,68.296560,0.0,1.0,1


In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121634 entries, 0 to 121633
Data columns (total 24 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   title                          121634 non-null  object 
 1   release_date                   121634 non-null  object 
 2   type                           121634 non-null  object 
 3   runtime                        121634 non-null  float64
 4   season_count                   121634 non-null  float64
 5   episode_count                  121634 non-null  float64
 6   genre                          121634 non-null  object 
 7   director                       121634 non-null  object 
 8   cast                           121634 non-null  object 
 9   country                        121634 non-null  object 
 10  language                       121634 non-null  object 
 11  production_company             121634 non-null  object 
 12  synopsis                      

In [12]:
df_test.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,...,imdb_rating,imdb_review_count,text_input,production_company_is_missing,cast_is_missing,director_is_missing,cast_award_score,director_top_score,actor_top_score,director_productivity_success
0,STRAW,2025,movie,105.0,1.0,1.0,"Thriller, Drama, Crime",Tyler Perry,"Taraji P. Henson, Sherri Shepherd, Teyana Tayl...",US,...,8.118,484,What will be her last straw? A devastatingly b...,0,0,0,0.0,0.0,0.0,9.0
1,Predator: Killer of Killers,2025,movie,85.0,1.0,1.0,"Animation, Action, Science Fiction",Dan Trachtenberg,"Lindsay LaVanchy, Louis Ozawa, Rick Gonzalez, ...",US,...,7.981,525,While three of the fiercest warriors in human ...,0,0,0,0.0,0.0,0.2,20.0
2,How to Train Your Dragon,2025,movie,125.0,1.0,1.0,"Action, Family, Fantasy",Dean DeBlois,"Mason Thames, Nico Parker, Gerard Butler, Nick...",US,...,8.000,256,"On the rugged isle of Berk, where Vikings and ...",0,0,0,0.0,0.0,0.0,0.0
3,Deep Cover,2025,movie,99.0,1.0,1.0,"Action, Comedy, Crime",Tom Kingsley,"Bryce Dallas Howard, Orlando Bloom, Nick Moham...",US,...,6.811,119,Kat is an improv comedy teacher beginning to q...,0,0,0,0.0,0.0,0.2,0.0
4,K.O.,2025,movie,84.0,1.0,1.0,"Action, Drama, Adventure",Antoine Blossier,"Ciryl Gane, Alice Belaïdi, Maleaume Paquin, Fo...",FR,...,6.660,119,A former fighter must find the missing son of ...,0,0,0,0.0,0.0,0.0,0.0


In [13]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1477 entries, 0 to 1476
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   title                          1477 non-null   object 
 1   release_date                   1477 non-null   int64  
 2   type                           1477 non-null   object 
 3   runtime                        1477 non-null   float64
 4   season_count                   1477 non-null   float64
 5   episode_count                  1477 non-null   float64
 6   genre                          1477 non-null   object 
 7   director                       1477 non-null   object 
 8   cast                           1477 non-null   object 
 9   country                        1477 non-null   object 
 10  language                       1477 non-null   object 
 11  production_company             1477 non-null   object 
 12  synopsis                       1477 non-null   o

In [15]:
df_train.shape

(121634, 24)

In [16]:
df_test.shape

(1477, 24)